#PREENCHA SEU NOME COMPLETO AQUI: 

### <span style="color:blue">MBA em Ciência de Dados</span>
### <span style="color:blue">Redes Neurais e Arquiteturas Profundas</span>
### <span style="color:blue">Análise de Dados com Base em Processamento Massivo em Paralelo</span>

### <span style="color:blue">Prova Final</span>

**Material Produzido por:**<br>
>**Profa. Dra. Cristina Dutra de Aguiar**<br>
>**Prof. Dr. Moacir A. Ponti**<br> 

**CEMEAI - ICMC/USP São Carlos**


A prova final contém 1 questão, dividida em 3 itens. Por favor, procurem por Questão para encontrar a especificação da questão e por RESOLVER para encontrar a especificação do item a ser solucionado. Também é possível localizar a questão e os itens utilizando o menu de navegação. 

O notebook contém a constelação de fatos da BI Solutions que deve ser utilizada para responder à questão e também todas as `bibliotecas`, `bases de dados`, `inicializações`, `instalações`, `importações`, `geração de dataFrames`, `geração de visões temporárias` e `conversão dos tipos de dados` necessárias para a realização da questão.


**INSTRUÇÕES**:<br>
1) Você deve exportar esse notebook com sua solução para as questões da prova em formato .py e fazer upload no Moodle. Atenção: você não deve fazer upload de um arquivo notebook (.ipynb), mas sim um arquivo texto .py contendo os códigos python que utilizou para resolver as questões. O arquivo .py pode ser gerado através da opção:<br>
File --> Download as --> Python (.py)
disponível no Jupyter Notebook.

ou
File --> Download .py
no Google Colab

Caso não esteja utilizando o Jupyter, copie e cole seu código em um arquivo ASCII (Texto) salvando com a extensão .py

2) Você deve salvar esse notebook com sua solução para as questões da prova em formato .pdf e fazer upload no Moodle

3) Os arquivos devem ser nomeados com seu nome e sobrenome, sem espaços. Exemplo: moacirponti.py e moacirponti.pdf

4) É OBRIGATÓRIO conter no cabeçalho (início) do arquivo um comentário / texto com o seu nome completo


**Desejamos uma boa prova!**

#1 Constelação de Fatos da BI Solutions

A aplicação de *data warehousing* da BI Solutions utiliza como base uma contelação de fatos, conforme descrita a seguir.

**Tabelas de dimensão**

- data (dataPK, dataCompleta, dataDia, dataMes, dataBimestre, dataTrimestre, dataSemestre, dataAno)
- funcionario (funcPK, funcMatricula, funcNome, funcSexo, funcDataNascimento, funcDiaNascimento, funcMesNascimento, funcAnoNascimento, funcCidade, funcEstadoNome, funcEstadoSigla, funcRegiaoNome, funcRegiaoSigla, funcPaisNome, funcPaisSigla)
- equipe (equipePK, equipeNome, filialNome, filialCidade, filialEstadoNome, filialEstadoSigla, filialRegiaoNome, filialRegiaoSigla, filialPaisNome, filialPaisSigla)
- cargo (cargoPK, cargoNome, cargoRegimeTrabalho, cargoEscolaridadeMinima, cargoNivel)
- cliente (clientePK, clienteNomeFantasia, clienteSetor, clienteCidade, clienteEstadoNome, clienteEstadoSigla, clienteRegiaoNome, clienteRegiaoSigla, clientePaisNome, clientePaisSigla)

**Tabelas de fatos**
- pagamento (dataPK, funcPK, equipePK, cargoPK, salario, quantidadeLancamentos)
- negociacao (dataPK, equipePK, clientePK, receita, quantidadeNegociacoes)


#2 Configurações 


## 2.1 Obtenção dos Dados da BI Solutions

In [ ]:
#instalando o módulo wget
%%capture
!pip install -q wget
!mkdir data

#baixando os dados das tabelas de dimensão e das tabelas de fatos
import wget

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/data.csv"
wget.download(url, "data/data.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/funcionario.csv"
wget.download(url, "data/funcionario.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/equipe.csv"
wget.download(url, "data/equipe.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cargo.csv"
wget.download(url, "data/cargo.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cliente.csv"
wget.download(url, "data/cliente.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/pagamento.csv"
wget.download(url, "data/pagamento.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/negociacao.csv"
wget.download(url, "data/negociacao.csv")

## 2.2 Instalações e Inicializações

In [ ]:
#instalando Java Runtime Environment (JRE) versão 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#baixando Apache Spark versão 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

In [ ]:
import os
#configurando a variável de ambiente JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [ ]:
%%capture
#instalando o pacote findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

## 2.3 Bibliotecas

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import round, desc

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from tensorflow.random import set_seed
from tensorflow import keras
from tensorflow.keras import layers

## 2.4 Geração dos DataFrames em Pandas da BI Solutions

Nesta seção são gerados os DataFrames em Pandas. Atenção aos nomes desses DataFrames. 

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
cargoPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cargo.csv')
clientePandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cliente.csv')
dataPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/data.csv')
equipePandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/equipe.csv')
funcionarioPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/funcionario.csv')
negociacaoPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/negociacao.csv')
pagamentoPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/pagamento.csv')

## 2.5 Geração dos DataFrames em Spark da BI Solutions

Nesta seção são gerados dos DataFrames em Spark. Atenção aos nomes desses DataFrames. 

In [ ]:
#criando os DataFrames em Spark 
cargo = spark.read.csv(path="data/cargo.csv", header=True, sep=",")
cliente = spark.read.csv(path="data/cliente.csv", header=True, sep=",")
data = spark.read.csv(path="data/data.csv", header=True, sep=",")
equipe = spark.read.csv(path="data/equipe.csv", header=True, sep=",")
funcionario = spark.read.csv(path="data/funcionario.csv", header=True, sep=",")
negociacao = spark.read.csv(path="data/negociacao.csv", header=True, sep=",")
pagamento = spark.read.csv(path="data/pagamento.csv", header=True, sep=",")

In [ ]:
#convertendo os dados necessários para o tipo de dado inteiro
colunas_cargo = ["cargoPK"]
colunas_cliente = ["clientePK"]
colunas_data = ["dataPK", "dataDia", "dataMes", "dataBimestre", "dataTrimestre", "dataSemestre", "dataAno"]
colunas_equipe = ["equipePK"]
colunas_funcionario = ["funcPK", "funcDiaNascimento", "funcMesNascimento", "funcAnoNascimento"]
colunas_negociacao = ["equipePK", "clientePK", "dataPK", "quantidadeNegociacoes"]
colunas_pagamento = ["funcPK", "equipePK", "dataPK", "cargoPK", "quantidadeLancamentos"]

for coluna in colunas_cargo:
  cargo = cargo.withColumn(coluna, cargo[coluna].cast(IntegerType()))

for coluna in colunas_cliente:
  cliente = cliente.withColumn(coluna, cliente[coluna].cast(IntegerType()))

for coluna in colunas_data:
  data = data.withColumn(coluna, data[coluna].cast(IntegerType()))

for coluna in colunas_equipe:
  equipe = equipe.withColumn(coluna, equipe[coluna].cast(IntegerType()))

for coluna in colunas_funcionario:
  funcionario = funcionario.withColumn(coluna, funcionario[coluna].cast(IntegerType()))

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(IntegerType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(IntegerType()))

In [ ]:
#convertendo os dados necessários para o tipo de dado float
colunas_negociacao = ["receita"]
colunas_pagamento = ["salario"]

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(FloatType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(FloatType()))

In [ ]:
#criando as visões temporárias 
cargo.createOrReplaceTempView("cargo")
cliente.createOrReplaceTempView("cliente")
data.createOrReplaceTempView("data")
equipe.createOrReplaceTempView("equipe")
funcionario.createOrReplaceTempView("funcionario")
negociacao.createOrReplaceTempView("negociacao")
pagamento.createOrReplaceTempView("pagamento")

# 3 Revisão



Liste, para cada trimestre de cada ano, a soma dos salários dos funcionários que moram no estado de São Paulo e que tenham cargo de nível SENIOR. Arredonde a soma dos salários para até duas casas decimais. Devem ser exibidas as colunas na ordem e com os nomes especificados a seguir: "TRIMESTRE", "ANO", "TOTALSALARIO". Ordene as linhas exibidas primeiro por trimestre, depois pelo ano, depois pelo total de salários, todos em ordem crescente (ascendente). Liste as primeiras 25 linhas da resposta, sem truncamento das strings.

## 3.1 Consulta em SQL 



In [ ]:
# Escreva aqui a sua resposta para a consulta OLAP
query = """
SELECT dataTrimestre AS TRIMESTRE,
       dataAno AS ANO,
       ROUND(SUM(salario), 2) As TOTALSALARIO
FROM pagamento JOIN data ON pagamento.dataPK = data.dataPK
               JOIN funcionario ON pagamento.funcPK = funcionario.funcPK
               JOIN cargo ON pagamento.cargoPK = cargo.cargoPK
WHERE funcEstadoNome = "SAO PAULO"
      AND cargoNivel = "SENIOR"               
GROUP BY dataTrimestre, dataAno
ORDER BY dataTrimestre, dataAno, TOTALSALARIO
"""
spark.sql(query).show(25,truncate=False)

+---------+----+------------+
|TRIMESTRE|ANO |TOTALSALARIO|
+---------+----+------------+
|1        |2016|272077.02   |
|1        |2017|656009.61   |
|1        |2018|1016968.39  |
|1        |2019|1155803.43  |
|1        |2020|1155803.43  |
|2        |2016|272077.02   |
|2        |2017|656009.61   |
|2        |2018|1016968.39  |
|2        |2019|1155803.43  |
|2        |2020|1155803.43  |
|3        |2016|272077.02   |
|3        |2017|656009.61   |
|3        |2018|1016968.39  |
|3        |2019|1155803.43  |
|3        |2020|1155803.43  |
|4        |2016|272077.02   |
|4        |2017|656009.61   |
|4        |2018|1016968.39  |
|4        |2019|1155803.43  |
|4        |2020|1155803.43  |
+---------+----+------------+



## 3.2 Consulta em Pandas

In [ ]:
# Escreva aqui a sua resposta para a consulta OLAP
mergePagamentoData = pagamentoPandas.merge(dataPandas, on= "dataPK")
mergePagamentoDataFuncionario = mergePagamentoData.merge(funcionarioPandas, on= "funcPK")
mergePagamentoDataFuncionarioCargo = mergePagamentoDataFuncionario.merge(cargoPandas, on= "cargoPK")
mergePagamentoDataFuncionarioCargo = mergePagamentoDataFuncionarioCargo.query("funcEstadoNome == 'SAO PAULO'")
mergePagamentoDataFuncionarioCargo = mergePagamentoDataFuncionarioCargo.query('cargoNivel == "SENIOR"')
df = mergePagamentoDataFuncionarioCargo.groupby(["dataTrimestre", "dataAno"], as_index=False)["salario"].sum().round(2)
df = df.sort_values(by=["dataTrimestre", "dataAno",  "salario"], ascending=True)
df = df.rename(columns={"dataTrimestre":"TRIMESTRE", "dataAno":"ANO", "salario":"TOTALSALARIO"})
display(df)



,TRIMESTRE,ANO,TOTALSALARIO
0,1,2016,272077.02
1,1,2017,656009.61
2,1,2018,1016968.38
3,1,2019,1155803.43
4,1,2020,1155803.43
5,2,2016,272077.02
6,2,2017,656009.61
7,2,2018,1016968.38
8,2,2019,1155803.43
9,2,2020,1155803.43


In [ ]:
# Escreva aqui a sua resposta para a consulta OLAP

# Listando apenas os dados de interesse
comparaFuncionario = funcionarioPandas.query("funcEstadoNome == 'SAO PAULO'")
comparaCargo = cargoPandas.query('cargoNivel == "SENIOR"') 

# Fazendo as junções 
mergePagamentoData = pagamentoPandas.merge(dataPandas, on= "dataPK")
mergePagamentoDataFuncionario = mergePagamentoData.merge(comparaFuncionario, on= "funcPK")
mergePagamentoDataFuncionarioCargo = mergePagamentoDataFuncionario.merge(comparaCargo, on= "cargoPK")
df = mergePagamentoDataFuncionarioCargo.groupby(["dataTrimestre", "dataAno"], as_index=False)["salario"].sum().round(2)
df = df.sort_values(by=["dataTrimestre", "dataAno",  "salario"], ascending=True)
df = df.rename(columns={"dataTrimestre":"TRIMESTRE", "dataAno":"ANO", "salario":"TOTALSALARIO"})
display(df)

,TRIMESTRE,ANO,TOTALSALARIO
0,1,2016,272077.02
1,1,2017,656009.61
2,1,2018,1016968.38
3,1,2019,1155803.43
4,1,2020,1155803.43
5,2,2016,272077.02
6,2,2017,656009.61
7,2,2018,1016968.38
8,2,2019,1155803.43
9,2,2020,1155803.43


## 3.3 Consulta em pyspark


In [ ]:
# Escreva aqui a sua resposta para a consulta OLAP
pagamento\
  .join(data, on="dataPK")\
  .join(funcionario, on='funcPK')\
  .join(cargo, on='cargoPK')\
  .where("funcEstadoNome == 'SAO PAULO'")\
  .where("cargoNivel == 'SENIOR'")\
  .groupBy('dataTrimestre', 'dataAno')\
  .sum('salario')\
  .withColumn('sum(salario)', round('sum(salario)', 2))\
  .withColumnRenamed('sum(salario)', 'TOTALSALARIO')\
  .withColumnRenamed('dataTrimestre', 'TRIMESTRE')\
  .withColumnRenamed('dataAno', 'ANO')\
  .orderBy('TRIMESTRE', 'ANO', 'TOTALSALARIO')\
  .show()

+---------+----+------------+
|TRIMESTRE| ANO|TOTALSALARIO|
+---------+----+------------+
|        1|2016|   272077.02|
|        1|2017|   656009.61|
|        1|2018|  1016968.39|
|        1|2019|  1155803.43|
|        1|2020|  1155803.43|
|        2|2016|   272077.02|
|        2|2017|   656009.61|
|        2|2018|  1016968.39|
|        2|2019|  1155803.43|
|        2|2020|  1155803.43|
|        3|2016|   272077.02|
|        3|2017|   656009.61|
|        3|2018|  1016968.39|
|        3|2019|  1155803.43|
|        3|2020|  1155803.43|
|        4|2016|   272077.02|
|        4|2017|   656009.61|
|        4|2018|  1016968.39|
|        4|2019|  1155803.43|
|        4|2020|  1155803.43|
+---------+----+------------+

